In [8]:
#Import dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
from sqlalchemy import create_engine, func
import sqlite3


#Connect to sqlite
connection = sqlite3.connect(":memory:")

cursor = connection.cursor()

#Read in sql file
sql_file = open("living_in_nashville.sql")
sql_as_string = sql_file.read()
cursor.executescript(sql_as_string)


In [43]:
#Create housing DataFrame
housing_list = []
for row in cursor.execute("SELECT * FROM Nashville_Housing_Data"):
    housing_list.append(row)

#housing_df
housing_df = pd.DataFrame(housing_list)
housing_df.columns = ['Unnamed:0',"ParcelID","LandUse","PropertyAddress","Suite/Condo#","PropertyCity","ZipCode","SaleDate","SalePrice","LegalReference","SoldAsVacant","MultipleParcelsInvolvedinSale","OwnerName","Address","City","State","Acreage","TaxDistrict","Neighborhood","image","LandValue","BuildingValue","TotalValue","FinishedArea","FoundationType","YearBuilt","ExteriorWall","Grade","Bedrooms","FullBath","HalfBath"]
housing_df.head()


,Unnamed:0,ParcelID,LandUse,PropertyAddress,Suite/Condo#,PropertyCity,ZipCode,SaleDate,SalePrice,LegalReference,...,BuildingValue,TotalValue,FinishedArea,FoundationType,YearBuilt,ExteriorWall,Grade,Bedrooms,FullBath,HalfBath
0,32,162 02 0A 008.00,SINGLE FAMILY,744 HILL BENNETT CIR,None,ANTIOCH,37013,1/7/2013,134825,20130108-0002492,...,None,None,None,None,None,None,None,None,None,None
1,33,162 06 0 012.00,SINGLE FAMILY,152 TUSCULUM RD,None,ANTIOCH,37013,1/15/2013,109900,20130124-0007776,...,77400,101400,1247,CRAWL,1968,BRICK,C,3,1,1
2,34,162 13 0A 015.00,SINGLE FAMILY,2321 GREEN TRAILS CT,None,ANTIOCH,37013,1/4/2013,152950,20130108-0002487,...,None,None,None,None,None,None,None,None,None,None
3,45,173 04 0A 202.00,SINGLE FAMILY,5224 RIDGEFALLS WAY,None,ANTIOCH,37013,1/18/2013,195000,20130122-0007001,...,None,None,None,None,None,None,None,None,None,None
4,46,173 04 0A 274.00,SINGLE FAMILY,5457 OAK CHASE DR,None,ANTIOCH,37013,1/18/2013,230000,20130128-0008839,...,None,None,None,None,None,None,None,None,None,None


In [76]:
# Create Parks DataFrame
parks_list = []
for row in cursor.execute("SELECT * FROM Park_Locations"):
    parks_list.append(row)

parks_df = pd.DataFrame(parks_list)
parks_df.columns= ["ParkName","ZipCodes","Acres","CommunityCenter","NatureCenter","Playground","ADAAccessible","RestroomsAvailable","DogPark","BaseballFields","BasketballCourts","SoccerFields","Football&Multi-purposeFields","TennisCourts","DiscGolf","SwimmingPool","GolfCourse","Walk&JogPaths","HistoricFeatures","MappedLocation","Lat","Lng"]
parks_df=parks_df.replace({'Yes':True,'No':False})
parks_df.head()

,ParkName,ZipCodes,Acres,CommunityCenter,NatureCenter,Playground,ADAAccessible,RestroomsAvailable,DogPark,BaseballFields,...,Football&Multi-purposeFields,TennisCourts,DiscGolf,SwimmingPool,GolfCourse,Walk&JogPaths,HistoricFeatures,MappedLocation,Lat,Lng
0,Antioch Park,37013,7.98,True,False,True,True,True,False,False,...,False,False,False,False,False,True,False,"5023 Blue Hole Rd Antioch, TN 37013",36.055271,-86.672392
1,Cane Ridge Park,37013,274.13,False,False,True,True,True,False,True,...,True,True,False,False,False,True,False,"419 Battle Rd Antioch, TN 37013",35.994965,-86.624342
2,Cecil Rhea Crawford Park,37013,6.25,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,"6043 Cane Ridge Rd Antioch, TN 37013",36.001427,-86.636764
3,Southeast Community Center Park,37013,12.14,True,False,True,True,True,False,False,...,False,False,False,False,False,True,False,"5260 Hickory Hollow Pkwy Antioch, TN 37013",36.048466,-86.657263
4,William A. Pitts Park,37013,26.94,False,False,True,True,False,True,True,...,False,False,False,False,False,False,False,"299 Tusculum Rd Nashville, TN",36.059541,-86.706819


In [ ]:
#Create list for of whether Walk and Jog is True
walk_list_per_zip = parks_df.groupby('ZipCodes')['Walk&JogPaths'].apply(list)
walk_per_zip = []
for code in walk_list_per_zip:
    if True in code:
        walk_per_zip.append(True)
    else: walk_per_zip.append(False)
walk_per_zip

In [93]:
#Create list for baseball
baseball_per_zip = parks_df.groupby('ZipCodes')['BaseballFields'].apply(list)
baseball_list = []
for code in baseball_per_zip:
    if True in code:
        baseball_list.append(True)
    else: baseball_list.append(False)
#Create list for basketball
basketball_per_zip = parks_df.groupby('ZipCodes')['BasketballCourts'].apply(list)
basketball_list = []
for code in basketball_per_zip:
    if True in code:
        basketball_list.append(True)
    else: basketball_list.append(False)
#Create list for Soccer
soccer_per_zip = parks_df.groupby('ZipCodes')['SoccerFields'].apply(list)
soccer_list = []
for code in soccer_per_zip:
    if True in code:
        soccer_list.append(True)
    else: soccer_list.append(False)
#Create list for Football
football_per_zip = parks_df.groupby('ZipCodes')['Football&Multi-purposeFields'].apply(list)
football_list = []
for code in football_per_zip:
    if True in code:
        football_list.append(True)
    else: football_list.append(False)
#Create list for Tennis
tennis_per_zip = parks_df.groupby('ZipCodes')['TennisCourts'].apply(list)
tennis_list = []
for code in tennis_per_zip:
    if True in code:
        tennis_list.append(True)
    else: tennis_list.append(False)

In [ ]:
#Zip together individual sport lists
sports_tuples = list(zip(baseball_list,basketball_list,soccer_list,football_list,tennis_list))
sports_tuples

In [ ]:
#Create list for whether any of the sports lists contain true in each zip
sports_list = []
for row in sports_tuples:
    if True in row:
        sports_list.append(True)
    else: sports_list.append(False)

sports_list

In [ ]:
#group by zip code and find if there is a yes in ADA column

ada_list_per_zip = parks_df.groupby('ZipCodes')['ADAAccessible'].apply(list)
ada_per_zip = []
for code in ada_list_per_zip:
    if True in code:
        ada_per_zip.append(True)
    else: ada_per_zip.append(False)
ada_per_zip

In [ ]:
# Create parks per zip code variable
parks_per_zip = parks_df.groupby('ZipCodes').size()
parks_per_zip

In [ ]:
# Create variable for total acres of parks per zip code
acres_per_zip = parks_df.groupby('ZipCodes').sum()['Acres']
acres_per_zip

In [108]:
#Create new DataFrame with created variables
parks_new_tuples = list(zip(parks_per_zip, acres_per_zip, ada_per_zip, sports_list, walk_per_zip))
parks_new_df = pd.DataFrame(parks_new_tuples, columns=["Parks Per Zip", "Acres of Parks Per Zip", "ADA Park in Zip", "Sports Fields in Zip", "Walk Path in Zip"], index=parks_per_zip.index)
parks_new_df

,Parks Per Zip,Acres of Parks Per Zip,ADA Park in Zip,Sports Fields in Zip,Walk Path in Zip
ZipCodes,,,,,
37013,5,327.44,True,True,True
37015,1,2168.79,False,False,False
37018,1,4.40,False,True,False
37027,1,6.95,True,True,True
37076,2,31.95,True,False,False
37080,2,116.79,True,True,True
37115,3,1001.91,True,True,True
37138,1,2.39,False,True,False
37189,1,19.70,False,True,False


In [111]:
# Merge new parks df with housing df
new_housing_df = housing_df.merge(parks_new_df, left_on="ZipCode", right_on="ZipCodes")
new_housing_df.head()

,Unnamed:0,ParcelID,LandUse,PropertyAddress,Suite/Condo#,PropertyCity,ZipCode,SaleDate,SalePrice,LegalReference,...,ExteriorWall,Grade,Bedrooms,FullBath,HalfBath,Parks Per Zip,Acres of Parks Per Zip,ADA Park in Zip,Sports Fields in Zip,Walk Path in Zip
0,32,162 02 0A 008.00,SINGLE FAMILY,744 HILL BENNETT CIR,None,ANTIOCH,37013,1/7/2013,134825,20130108-0002492,...,None,None,None,None,None,5,327.44,True,True,True
1,33,162 06 0 012.00,SINGLE FAMILY,152 TUSCULUM RD,None,ANTIOCH,37013,1/15/2013,109900,20130124-0007776,...,BRICK,C,3,1,1,5,327.44,True,True,True
2,34,162 13 0A 015.00,SINGLE FAMILY,2321 GREEN TRAILS CT,None,ANTIOCH,37013,1/4/2013,152950,20130108-0002487,...,None,None,None,None,None,5,327.44,True,True,True
3,45,173 04 0A 202.00,SINGLE FAMILY,5224 RIDGEFALLS WAY,None,ANTIOCH,37013,1/18/2013,195000,20130122-0007001,...,None,None,None,None,None,5,327.44,True,True,True
4,46,173 04 0A 274.00,SINGLE FAMILY,5457 OAK CHASE DR,None,ANTIOCH,37013,1/18/2013,230000,20130128-0008839,...,None,None,None,None,None,5,327.44,True,True,True


In [47]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np



In [ ]:
# Split our preprocessed data into our features and target arrays
y = df[target_variable].values
X = df.drop(target_variable,1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
#Create a linear regression model 
from sklearn.linear_model import LinearRegression
classifier = LinearRegression()
classifier

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Fitting our model with all our features in X
model.fit(X, y)

score = model.score(X, y)
print(f"R2 Score: {score}")

In [ ]:
predictions = model.predict(X)

In [ ]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
#Create Schools DataFrame
schools_list = []
for row in cursor.execute("SELECT * FROM MNPS_Enrollment_Data"):
    schools_list.append(row)


schools_df = pd.DataFrame(schools_list)
schools_df.columns = ["SchoolLevel","SchoolName","ZipCode","Rank","Greatschoolsrating","TotalEnrollment","AmericanIndianorAlaskaNative","Asian","BlackorAfricanAmerican","Hispanic/Latino","NativeHawaiianorOtherPacificIslander","White","StudentswithDisabilities"]
schools_df.head()

In [ ]:
#Create Schools DataFrame
schools_list = []
for row in cursor.execute("SELECT * FROM MNPS_Enrollment_Data"):
    schools_list.append(row)


schools_df = pd.DataFrame(schools_list)
schools_df.columns = ["SchoolLevel","SchoolName","ZipCode","Rank","Greatschoolsrating","TotalEnrollment","AmericanIndianorAlaskaNative","Asian","BlackorAfricanAmerican","Hispanic/Latino","NativeHawaiianorOtherPacificIslander","White","StudentswithDisabilities"]
schools_df.head()